https://facebook.github.io/prophet/docs/quick_start.html

# 라이브러리 불러오기

In [1]:
# Python
import pandas as pd
from prophet import Prophet
from prophet.plot import plot_components_plotly
from prophet.serialize import model_to_json
from sklearn.metrics import mean_squared_error
from src.app.app_utils import plot_plotly_with_testset

/Users/jeanboy/workspace/time-series-ml-app/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 데이터 불러오기

In [2]:
# Python
df = pd.read_csv('https://raw.githubusercontent.com/facebook/prophet/main/examples/example_wp_log_peyton_manning.csv')
df.head()


,ds,y
0,2007-12-10,9.590761
1,2007-12-11,8.519590
2,2007-12-12,8.183677
3,2007-12-13,8.072467
4,2007-12-14,7.893572


In [3]:
df.tail()

,ds,y
2900,2016-01-16,7.817223
2901,2016-01-17,9.273878
2902,2016-01-18,10.333775
2903,2016-01-19,9.125871
2904,2016-01-20,8.891374


# 학습, 평가 데이터 분리

In [4]:
df['ds'] = pd.to_datetime(df.ds)
df_train = df[df.ds<"2015-01-01"]
df_test = df[(df.ds>="2015-01-01")&(df.ds<"2016-01-01")]

# 학습

In [5]:
m = Prophet()
m.fit(df_train)

01:38:10 - cmdstanpy - INFO - Chain [1] start processing
01:38:10 - cmdstanpy - INFO - Chain [1] done processing


# 평가

In [6]:
# 예측
future = m.make_future_dataframe(periods=365, include_history=False)
forecast = m.predict(future)

# metric 계산
results = pd.merge(
    df_test[['ds','y']],
    forecast[['ds','yhat']],
    on='ds', how="left"
    )
rmse_ = mean_squared_error(results.y, results.yhat)
print(rmse_)

0.2866715466703453


# 예측 결과 시각화

In [7]:
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
2882,2015-12-27
2883,2015-12-28
2884,2015-12-29
2885,2015-12-30
2886,2015-12-31


In [8]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2882,2015-12-27,8.464362,7.697124,9.201697
2883,2015-12-28,8.762475,8.052491,9.503127
2884,2015-12-29,8.545231,7.811844,9.273852
2885,2015-12-30,8.380259,7.613861,9.151672
2886,2015-12-31,8.397520,7.677742,9.160066


In [9]:
plot_plotly_with_testset(m, forecast,future_real=df_test)

In [10]:
plot_components_plotly(m,forecast)

# 모델 저장

In [11]:
# Python
with open('../src/app/prophet_model.json', 'w') as fout:
    fout.write(model_to_json(m))  # Save model